# <b> Carga de librerias: </b>

In [1]:
import pandas as pd 
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
import modules
import preprocessors as pp 
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler #Estandarizador

# <b>Carga de datos:</b>

In [2]:
datos = pd.read_csv('dataset_proyecto_analisis.csv')
datos.head()

,BUCKET,DIAS_MORA,DIAS_INCUMPLIMIENTO,MET_CALCULO,MARCA_DEFAULT,PDI,RANGO_SCORE_PN,seg_PD12mes,PD_12M_ACTUAL_FINAL,PD_12M_DESEMBOLSO_FINAL,PD_LIFETIME_ACTUAL_FINAL,PD_LIFETIME_DESEMBOLSO_FINAL,EXPOSICION_TOTAL_FA,VALOR_PROVISION,cobertura,target
0,1,0,NaN,ECL_12M,0,0.50798,1.0,PN_CURA,0.325910,0.325910,0.304117,0.066976,3142.9200,520.328516,0.165556,Al dia
1,1,0,NaN,ECL_12M,0,0.50798,2.0,PN_CURA,0.425376,0.425376,0.432310,0.220604,33364.3717,7956.298017,0.238467,Al dia
2,1,0,NaN,ECL_12M,0,0.50798,3.0,PN_CURA,0.445730,0.445730,0.457113,0.245921,106599.3731,26970.279477,0.253006,Al dia
3,1,0,NaN,ECL_12M,0,0.50798,4.0,PN_CURA,0.434547,0.434547,0.455007,0.257770,137366.7897,34997.765487,0.254776,Al dia
4,1,0,NaN,ECL_12M,0,0.50798,5.0,PN_CURA,0.343117,0.343117,0.404523,0.217305,95085.2122,23860.250873,0.250935,Al dia


# <b> Verificación de valores nulos </b>:
Con base a esta inspección se aplicara la imputación <b>CCA</b> para las variables valores <br>
nulos menores al 0.05 y para la DIAS_INCUMPLIMIENTO se aplicara la imputación <br> 
<b> Arbitrary_Imputation </b>, procedimientos que seran aplicados en el Pipeline

In [3]:
datos.isnull().mean(), datos.shape

(BUCKET                          0.000000
 DIAS_MORA                       0.000000
 DIAS_INCUMPLIMIENTO             0.144866
 MET_CALCULO                     0.000000
 MARCA_DEFAULT                   0.000000
 PDI                             0.000000
 RANGO_SCORE_PN                  0.000067
 seg_PD12mes                     0.000067
 PD_12M_ACTUAL_FINAL             0.000000
 PD_12M_DESEMBOLSO_FINAL         0.000000
 PD_LIFETIME_ACTUAL_FINAL        0.000000
 PD_LIFETIME_DESEMBOLSO_FINAL    0.000000
 EXPOSICION_TOTAL_FA             0.000000
 VALOR_PROVISION                 0.000000
 cobertura                       0.000269
 target                          0.000000
 dtype: float64,
 (14862, 16))

# <b> Pipeline preprocesador de datos </b>

In [4]:
def instanciaPipeline(dataset, arbitrary, var_y):
    
    nums_cols_na = modules.getNanGoodCols(df=dataset, rate=0.05)
    categoric_vars, discrete_vars, continues_vars = modules.getColumnsDataTypes(df=dataset)
    categoric_vars.remove(var_y)

    outliers_vars = categoric_vars + discrete_vars + continues_vars

    feature_engineering_pipeline = Pipeline(steps=[
        ('Complete_Case_Analysis', 
            pp.CompleteCaseAnalysis(varNames=nums_cols_na)
        ),
        ('Arbitrary_Imputation', 
            pp.ArbitraryImputation(varNames=arbitrary)
        ),
        ('Categorical_Encoder', 
            pp.CategoricalEncoderOperator(varNames=categoric_vars)
        ),
        ('OutliersTreatmentOperator', 
            pp.OutliersTreatmentOperator(varNames=outliers_vars)
        )

    ])

    return feature_engineering_pipeline

procesed_data = instanciaPipeline(dataset=datos, arbitrary='DIAS_INCUMPLIMIENTO', var_y='target').fit_transform(datos)
procesed_data['target'] = pd.get_dummies(procesed_data['target'], drop_first=True)

procesed_data.to_csv('dataset_proyecto_FE.csv', index=False)

categoric_vars, discrete_vars, continues_vars = modules.getColumnsDataTypes(df=datos)
var_transf = discrete_vars + continues_vars
modules.transformations(procesed_data, var_transf, 'target')
scaler = StandardScaler().fit(procesed_data.values)


c:\Users\apatz\anaconda3\lib\site-packages\numpy\core\_methods.py:243: RuntimeWarning: overflow encountered in reduce
  ret = umr_sum(x, axis, dtype, out, keepdims=keepdims, where=where)
c:\Users\apatz\anaconda3\lib\site-packages\numpy\core\_methods.py:232: RuntimeWarning: overflow encountered in multiply
  x = um.multiply(x, x, out=x)
c:\Users\apatz\anaconda3\lib\site-packages\scipy\optimize\optimize.py:2149: RuntimeWarning: invalid value encountered in double_scalars
  tmp2 = (x - v) * (fx - fw)
c:\Users\apatz\anaconda3\lib\site-packages\numpy\core\_methods.py:243: RuntimeWarning: overflow encountered in reduce
  ret = umr_sum(x, axis, dtype, out, keepdims=keepdims, where=where)
c:\Users\apatz\anaconda3\lib\site-packages\numpy\core\_methods.py:232: RuntimeWarning: overflow encountered in multiply
  x = um.multiply(x, x, out=x)
c:\Users\apatz\anaconda3\lib\site-packages\numpy\core\_methods.py:243: RuntimeWarning: overflow encountered in reduce
  ret = umr_sum(x, axis, dtype, out, keep

In [5]:
procesed_data.to_csv('dataset_proyecto_FE_2.csv', index=False)